
# Auto-Unclog Drain System with Python
In this notebook, I'll walk through the step-by-step process of creating a simulation for an auto-unclog drain system using Python fundamentals. The goal is to simulate a smart drain that detects water, monitors drain rates, and unclogs the drain if necessary.

We'll explore basic programming concepts, such as conditionals, loops, and functions, as well as more complex ideas like state monitoring and automated actions. Let's get started!

## Step 1: Define the Basic Drain Functions
The first step is to define basic functions that check the drain's status, the presence of water, and simulate air compressions for unclogging.
    

In [3]:

# Function to check if water is present in the tub (water detection sensor)
def is_water_present():
    return water_sensor.detect_water()  # Simulated water detection

# Function to check if the drain is stopped intentionally (i.e. for a bath)

def is_drain_stopped():
    return drain_sensor.is_stopped()  # Simulated drain stop detection
    


## Step 2: Implement the Drain Compression Logic
Once we detect that water isn't draining properly, we simulate drain compressions (three pushes with natural pullbacks) to unclog the drain. Here's how we implement that:
    

In [4]:

# Function to simulate a single drain compression (three pushes, with natural pullbacks)
def drain_compression():
    print("Starting drain compression...")
    for i in range(3):  # Three pushes
        print(f"Push {i+1}")
        air_pump.push_air()  # Simulate pushing air into the drain
        print("Returning to start position (natural pull)")
        air_pump.return_to_start()  # Natural return (acting as a pull)
    


## Step 3: Create the Baseline Drain Rate Test
Next, we define a baseline drain rate by measuring how fast water drains in an unclogged state. This will be compared against future drains to detect when a clog might be forming.
    

In [5]:

# Function to perform an initial drain rate test to establish a baseline
def perform_baseline_test():
    print("Starting baseline drain rate test...")
    fill_tub_to_max()  # Fill the tub to a specified max level
    start_time = time.now()
    open_drain()  # Start draining the tub
    while is_water_present():  # Wait until the water has drained completely
        pass
    end_time = time.now()
    baseline_drain_rate = calculate_drain_rate(start_time, end_time)  # Calculate drain rate
    print(f"Baseline drain rate established: {baseline_drain_rate} L/s")
    return baseline_drain_rate

# Function to calculate the drain rate (liters per second)
def calculate_drain_rate(start_time, end_time):
    tub_capacity = get_tub_capacity()  # Simulated tub capacity in liters
    drain_duration = end_time - start_time  # Calculate time taken to drain
    return tub_capacity / drain_duration.total_seconds()  # Drain rate in liters per second
    


## Step 4: Compare the Current Drain Rate to the Baseline
Now that we have a baseline, every time the drain is used, we can compare the current drain rate to the baseline and decide if the drain needs unclogging or maintenance.
    

In [6]:

# Function to check if the current drain rate is below the baseline
def is_drain_rate_below_baseline(current_drain_rate, baseline_drain_rate):
    return current_drain_rate < (baseline_drain_rate * 0.8)  # Consider 80% as threshold

# Function to check the drain rate and suggest maintenance if needed
def check_drain_rate(baseline_drain_rate):
    if is_water_present():
        print("Monitoring current drain rate...")
        start_time = time.now()
        open_drain()  # Start draining
        while is_water_present():
            pass
        end_time = time.now()
        current_drain_rate = calculate_drain_rate(start_time, end_time)
        print(f"Current drain rate: {current_drain_rate} L/s")
        
        if is_drain_rate_below_baseline(current_drain_rate, baseline_drain_rate):
            print("Drain rate is below baseline. Suggesting maintenance.")
            notify_user("Drain rate has dropped. Consider maintenance.")
        else:
            print("Drain rate is normal.")
    else:
        print("No water detected.")
    


## Step 5: Final Integration of the Unclog Logic and System Process
We now integrate the baseline check, water detection, and drain compression into one system to monitor and maintain the drain, unclogging it automatically when necessary.
    

In [7]:

# Main function to control the auto-unclog process
def auto_drain_unclog():
    baseline_drain_rate = perform_baseline_test()  # Establish baseline during setup
    
    if is_water_present():
        if not is_drain_stopped():  # If drain is not stopped (e.g., for a bath)
            check_drain_rate(baseline_drain_rate)  # Compare current drain rate to baseline
            if is_water_pooling():  # If water pools, indicating a clog
                print("Water pooling detected. Starting unclog process.")
                drain_compressions()  # Run compressions if needed
        else:
            print("Drain is stopped. Standby mode activated.")
    else:
        print("No water detected.")
    